In [1]:
!pip -q install mlflow transformers boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.6/752.6 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.6 MB/s eta 0:00:00


# Set up

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
import mlflow
import boto3
from threading import Thread
import time

In [3]:
# mlflow set tracking
url = "https://victoria-communicable-sometimes.ngrok-free.dev"
mlflow.set_tracking_uri(url)
tracking_uri = mlflow.get_tracking_uri()
print(f"Current tracking uri: {tracking_uri}")

Current tracking uri: https://victoria-communicable-sometimes.ngrok-free.dev


In [4]:
mlflow.set_experiment("healthcarechatbot")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1761320025344, experiment_id='1', last_update_time=1761320025344, lifecycle_stage='active', name='healthcarechatbot', tags={}>

In [5]:
from mlflow.tracking import MlflowClient
import re

client = MlflowClient()
model_name = "health-llm"
versions = client.get_latest_versions(model_name)

for v in versions:
    path = v.source
    print("Model URI:", path)

    match = re.match(r"s3://([^/]+)/(.*)", path)
    if match:
        bucket = match.group(2)
        print("Bucket:", bucket)
    else:
        print("Invalid model URI format")

/tmp/ipython-input-2271236965.py:6: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(model_name)


Model URI: s3://mlflow-artifacts-monitor/models/health-llm/257cf75b54b449aeb1014c6b480d7bae
Bucket: models/health-llm/257cf75b54b449aeb1014c6b480d7bae


# Model Downloading

In [6]:
import os
import boto3
from tqdm import tqdm
from dotenv import load_dotenv

def load_model_from_s3(s3_prefix: str, local_dir: str = "downloaded_model"):
    """
    Download an entire model directory (e.g., from MLflow-registered S3 prefix).
    Example s3_prefix: "models/health-llm/b3f91d2b6f42464aab9b9ff07d22ad89"
    """

    load_dotenv()

    aws_access_key = os.getenv("AWS_ACCESS_KEY_ID")
    aws_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
    aws_region = os.getenv("AWS_DEFAULT_REGION", "ap-southeast-2")
    bucket_name = os.getenv("AWS_BUCKET_NAME", "mlflow-artifacts-monitor")

    if not all([aws_access_key, aws_secret_key, bucket_name]):
        raise ValueError("Missing AWS credentials or bucket name in .env file")

    s3 = boto3.client(
        "s3",
        aws_access_key_id=aws_access_key,
        aws_secret_access_key=aws_secret_key,
        region_name=aws_region
    )

    os.makedirs(local_dir, exist_ok=True)

    paginator = s3.get_paginator("list_objects_v2")
    total_files = 0

    # Đếm file trước (để tqdm chạy đẹp)
    for page in paginator.paginate(Bucket=bucket_name, Prefix=s3_prefix):
        for obj in page.get("Contents", []):
            total_files += 1

    with tqdm(total=total_files, desc=f"Downloading model from {s3_prefix}") as pbar:
        for page in paginator.paginate(Bucket=bucket_name, Prefix=s3_prefix):
            for obj in page.get("Contents", []):
                key = obj["Key"]
                local_path = os.path.join(local_dir, os.path.relpath(key, s3_prefix))
                os.makedirs(os.path.dirname(local_path), exist_ok=True)

                s3.download_file(bucket_name, key, local_path)
                pbar.update(1)

    print(f"Model downloaded successfully → {local_dir}")
    return local_dir


In [7]:
local_dir = load_model_from_s3(bucket)

Model downloaded successfully → downloaded_model


# Model Inference

In [8]:
# Load tokenizer và model từ local
tokenizer = AutoTokenizer.from_pretrained(local_dir)
model = AutoModelForCausalLM.from_pretrained(local_dir)

In [9]:
question = "Dựa vào Điều 59 của Luật Hàng không dân dụng Việt Nam, chức năng chính của Cảng vụ hàng không là gì?"

## Batch predict

In [10]:
prompt = [
    {"role": "system", "content": "Bạn là một trợ lý luật pháp Việt Nam thông minh, luôn trả lời bằng tiếng Việt chuẩn và dễ hiểu."},
    {"role": "user", "content": question}
]

prompt = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

start = time.time()
outputs = model.generate(
    **inputs,
    max_new_tokens=384,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)
durations = time.time() - start

print(f"Time: {durations:.3f} giây")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Time: 109.184 giây

Bạn là một trợ lý luật pháp Việt Nam thông minh, luôn trả lời bằng tiếng Việt chuẩn và dễ hiểu.
 
 
Dựa vào Điều 59 của Luật Hàng không dân dụng Việt Nam, chức năng chính của Cảng vụ hàng không là gì?
 
 
Chức năng chính của Cảng vụ hàng không là tổ chức đơn vị hành vi và hệ quốc tế, tổ chức hoặc xã sâu người đồng hành quyền trên đất nước, cần có thêm thông tin sau: Thời điểm hán hành và số công viêctao phát sinh ra; thời điểm hán hành và mức độ cho vai trò của họ; nghiệp vụ hành tác nguyên liệu đồng hữu, để an toàn đem lao đặt chúng.


## Streaming Predict

In [11]:
streamer = TextIteratorStreamer(tokenizer, skip_special_tokens=True)

generation_kwargs = dict(
    **inputs,
    max_new_tokens=384,
    do_sample=True,
    temperature=0.7,
    top_k=50,
    top_p=0.9,
    repetition_penalty=1.2,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id,
    streamer=streamer
)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()

print("Assistant:", end=" ", flush=True)
for new_text in streamer:
    print(new_text, end="", flush=True)
thread.join()

Assistant: 
Bạn là một trợ lý luật pháp Việt Nam thông minh, luôn trả lời bằng tiếng Việt chuẩn và dễ hiểu.
 
 
Dựa vào Điều 59 của Luật Hàng không dân dụng Việt Nam, chức năng chính của Cảng vụ hàng không là gì?
 
 
Chúng ta có quyền số nghiên cứu cơ sở và khoản tâm đồng với đường kinh doanh ngân hàng, công ty hoặc khu vực cần biết để mang lại tác giả, viên học, yếu tố kinh tế, vận tải, vận hành của cảng cấp hàng không dân dụng Việt Nam.

# Clone llama.cpp

In [12]:
!git clone https://github.com/ggml-org/llama.cpp
!pip -q install -r llama.cpp/requirements.txt

Cloning into 'llama.cpp'...
remote: Enumerating objects: 65617, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 65617 (delta 4), reused 3 (delta 2), pack-reused 65603 (from 2)
Receiving objects: 100% (65617/65617), 183.62 MiB | 17.88 MiB/s, done.
Resolving deltas: 100% (47716/47716), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 61.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 7.5 MB/s 

In [13]:
!ls llama.cpp

AUTHORS			       examples    poetry.lock
build-xcframework.sh	       flake.lock  pyproject.toml
ci			       flake.nix   pyrightconfig.json
cmake			       ggml	   README.md
CMakeLists.txt		       gguf-py	   requirements
CMakePresets.json	       grammars    requirements.txt
CODEOWNERS		       include	   scripts
common			       LICENSE	   SECURITY.md
CONTRIBUTING.md		       licenses    src
convert_hf_to_gguf.py	       Makefile    tests
convert_hf_to_gguf_update.py   media	   tools
convert_llama_ggml_to_gguf.py  models	   vendor
convert_lora_to_gguf.py        mypy.ini
docs			       pocs


In [17]:
!python3 "llama.cpp/convert_hf_to_gguf.py"  ./downloaded_model --outfile  ./model.gguf --outtype q8_0

INFO:hf-to-gguf:Loading model: downloaded_model
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:hf-to-gguf:gguf: indexing model part 'model.safetensors'
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:output.weight,               torch.float32 --> Q8_0, shape = {2048, 32003}
INFO:hf-to-gguf:token_embd.weight,           torch.float32 --> Q8_0, shape = {2048, 32003}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.float32 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.ffn_down.weight,       torch.float32 --> Q8_0, shape = {5632, 2048}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float32 --> Q8_0, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float32 --> Q8_0, shape = {2048, 5632}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,       torch.float32 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float32 --> Q8_0, shape = {2048, 256}
INFO:hf-to-gguf:blk.0.attn

In [15]:
!pip -q install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 542.6 kB/s eta 0:00:00


## Inference

In [18]:
from llama_cpp import Llama

llm = Llama(model_path="./model.gguf",  verbose=False)

llama_context: n_ctx_per_seq (512) < n_ctx_train (2048) -- the full capacity of the model will not be utilized


In [19]:
prompt = [
    {"role": "system", "content": "Bạn là một trợ lý luật pháp Việt Nam thông minh, luôn trả lời bằng tiếng Việt chuẩn và dễ hiểu."},
    {"role": "user", "content": question}
]
start = time.time()

generation_options = {
    "max_tokens": 384,
    "temperature": 0.7,
    "top_p": 0.9,
    "top_k": 40,
    "stop": None,
}

response = llm.create_chat_completion(messages=prompt, **generation_options)
durations = time.time() - start

print(response['choices'][0]['message']['content'])
print(f"Time: {durations:.3f} giây")

Cảng vụ hàng không có được chính của Cảng, chức năng của Cảng vụ hàng không là gì.
Time: 21.154 giây


## Streaming

In [20]:
prompt = [
    {"role": "system", "content": "Bạn là một trợ lý luật pháp Việt Nam thông minh, luôn trả lời bằng tiếng Việt chuẩn và dễ hiểu."},
    {"role": "user", "content": question}
]

generation_options = {
    "max_tokens": 256,
    "temperature": 0.7,
    "top_p": 0.9,
    "top_k": 40,
    "stop": None,
}

for chunk in llm.create_chat_completion(messages=prompt, stream=True, **generation_options):
    choice = chunk['choices'][0]['delta']
    if 'role' in choice:
        print(f"\n[{choice['role']}]: ", end='', flush=True)
    if 'content' in choice:
        print(choice['content'], end='', flush=True)


[assistant]: Dựa vào Điều 59 của Luật Hàng không dân dụng Việt Nam, chức năng chính của Cảng vụ hàng không là gì là một thuật ngữ của Cảng vụ hàng không, làm tốt khi Cảng vụ hàng không có cơ sở sửa lọc, có cơ hội kết nối với cơ quan tư vấn và tỉnh thành hoặc khác, có cơ hội bảo trì và cơ hội đăng ký. Chức năng chính của Cảng vụ hàng không là gì là cơ cấu chính của Cảng vụ hàng không, làm tốt khi Cảng v

# Mlflow registry model

In [21]:
with mlflow.start_run() as run:

    s3_client = boto3.client(
        "s3",
        aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
        aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
        region_name=os.environ["AWS_DEFAULT_REGION"]
    )

    bucket = "mlflow-artifacts-monitor"
    s3_prefix = f"models/health-llm/{run.info.run_id}"


    model_path = "gguf_model.gguf"
    key = f"{s3_prefix}/{model_path}"
    s3_client.upload_file(model_path, bucket, key)

    model_uri = f"s3://{bucket}/{s3_prefix}"

    REGISTERED_MODEL_NAME = "health-llm-gguf"

    result = mlflow.register_model(
        model_uri=model_uri,
        name=REGISTERED_MODEL_NAME
    )

    client = MlflowClient()

    client.set_registered_model_tag(
        name=REGISTERED_MODEL_NAME, key="use_case", value="patient_service"
    )

    client.update_registered_model(
        name=REGISTERED_MODEL_NAME,
        description="A health-specific chatbot about daily Vietnamese sickness questions"
    )

    client.set_model_version_tag(
        name=REGISTERED_MODEL_NAME,
        version=result.version,
        key="validation_status",
        value="testing",
    )

    client.set_registered_model_alias(
        name=REGISTERED_MODEL_NAME,
        alias="champion",
        version=result.version,
    )

    print(f"Model registered successfully: version {result.version}")
    print(f"S3 path: {model_uri}")
    print(f"MLflow tracking: {run.info.run_id}")

Successfully registered model 'health-llm-gguf'.
2025/10/25 11:02:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: health-llm-gguf, version 1
Created version '1' of model 'health-llm-gguf'.


Model registered successfully: version 1
S3 path: s3://mlflow-artifacts-monitor/models/health-llm/3edc1eb810bb40728fb7962a42d61530
MLflow tracking: 3edc1eb810bb40728fb7962a42d61530
🏃 View run nervous-wren-355 at: https://victoria-communicable-sometimes.ngrok-free.dev/#/experiments/1/runs/3edc1eb810bb40728fb7962a42d61530
🧪 View experiment at: https://victoria-communicable-sometimes.ngrok-free.dev/#/experiments/1
